In [1]:
!pip install seaborn --quiet

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000

<br> 

# AUXILIAR FUNCTIONS

In [3]:
def recategorize(df, title, category_new):
    """Given a value passed in title_value, rename its category."""

    idxs = list()
    if isinstance(title, str):
        title = [title]
        
    for item in title:
        for idx, row in df.iterrows():
            if item.lower() in row['title'].lower():
                idxs.append(idx)
        df.loc[idxs, 'category'] = category_new
    
    return df

In [4]:
def group_categories(df, categories, category_new):
    idxs = list()
    for category in categories:
        aux = df[df['category'] == category]
        idxs = np.append(idxs, aux.index.values)
    df.loc[idxs, 'category'] = category_new
    
    return df

In [5]:
def get_categories_expenses(df):
    categories = []
    for category in df['category'].unique().tolist():
        categories.append([category, df[df['category'] == category].amount.sum()])

    categories = pd.DataFrame(categories, columns=['category', 'sum']).set_index('category')
    
    return categories['sum'].sort_values(ascending=False)

In [6]:
def add_expense_manually(df, title, amount, category, date='02/10/2020'):
    return df.append(pd.DataFrame(
                        {"date": [date], 
                         "category": [category],  
                         "title": [title],
                         "amount": [amount]}), ignore_index=True)

# Set attributes

In [11]:
month = '2021-11'

<br>

# GET DATA

### Get nubank

In [12]:
nubank = pd.read_csv('faturas/nubank-{}.csv'.format(month)).dropna()

<br> 

# COSTS

### > Raw categories

In [16]:
get_categories_expenses(nubank)

category
contas          2595.00
restaurante     1032.34
serviços         898.21
supermercado     795.11
eletrônicos      649.05
casa             529.53
transporte       489.70
viagem           465.00
outros           270.73
vestuário        211.36
saúde            156.48
lazer             47.27
educação           9.90
Name: sum, dtype: float64

### > Add expenses paid using debit

In [15]:
nubank = add_expense_manually(df=nubank, title='Aluguel+Condominio+água', amount=2000, category='contas')
nubank = add_expense_manually(df=nubank, title='Luz', amount=200, category='contas')
nubank = add_expense_manually(df=nubank, title='Internet', amount=105, category='contas')
nubank = add_expense_manually(df=nubank, title='Gás', amount=50, category='contas')
nubank = add_expense_manually(df=nubank, title='Ervas', amount=110, category='contas')
nubank = add_expense_manually(df=nubank, title='Gás', amount=50, category='contas')
nubank = add_expense_manually(df=nubank, title='Disney+', amount=10, category='contas')

### * CHECK HERE *

Check if categories are correct and adjust the wrongly categorized expenses

In [20]:
get_categories_expenses(nubank)

category
contas          2595.00
restaurante     1032.34
serviços         898.21
supermercado     795.11
eletrônicos      649.05
casa             529.53
transporte       489.70
viagem           465.00
outros           270.73
vestuário        211.36
saúde            156.48
lazer             47.27
educação           9.90
Name: sum, dtype: float64

In [29]:
# Check categories
nubank[nubank['category'] == 'educação']

,date,category,title,amount
18,2021-09-27,educação,Amazonprimebr,9.9


### > Recategorize wrong categorized purchases

In [30]:
nubank = recategorize(df=nubank, 
                      title=['farmacia', 'Gympass'], 
                      category_new='saúde')
nubank = recategorize(df=nubank, 
                      title=['Mercadolivre', 'Mercpago', 'Mercadolibre',
                             'Ame Digital', 'Amazon.com.br', 'Mercpag'],
                      category_new='novas_compras')
nubank = recategorize(df=nubank, 
                      title=['Nutrindus', 'pizzaria', 'restaurante', 'eats', 'Rappi'], 
                      category_new='restaurante') 
nubank = recategorize(df=nubank, 
                      title=['Amazonprimebr', 'Apple.Com/Bi', 
                             'Iugu *Contabili', 'Pb *Samsung', 
                             'Picpay'], 
                      category_new='contas')
nubank = recategorize(df=nubank, 
                      title=['Sympla', 
                             'Picpay'], 
                      category_new='lazer')
nubank = recategorize(df=nubank, 
                      title=['boteco', 'cervejaria', ' bar', 'growleria', 'Keromaisbebidas', 'Vino'], 
                      category_new='bar')
nubank = recategorize(df=nubank, 
                      title=['Sonyplaysta', 'Steamgames.Com'], 
                      category_new='videogames')
nubank = recategorize(df=nubank, 
                      title=['racoes', 'Ifd*Ifood', 'Petz'], 
                      category_new='supermercado')

### > Group categories

In [31]:
nubank = group_categories(df=nubank, 
                          categories=['outros', 'vestuário', 'lazer',
                                      'eletrônicos', 'casa', 'educação'], 
                          category_new='novas_compras')

### * CHECK AGAIN *

Check if categories are correct and adjust the wrongly categorized expenses

In [32]:
get_categories_expenses(nubank)

category
contas           2604.90
novas_compras    2131.33
restaurante      1032.34
supermercado      953.90
transporte        489.70
viagem            465.00
saúde             236.28
serviços          236.23
Name: sum, dtype: float64

In [37]:
# Check categories
nubank[nubank['category'] == 'transporte']

,date,category,title,amount
22,2021-09-30,transporte,Uber *Uber *Trip,26.94
23,2021-10-02,transporte,Uber *Uber *Trip,8.90
24,2021-10-04,transporte,Uber *Uber *Trip,14.93
25,2021-10-04,transporte,Uber *Uber *Trip,4.00
26,2021-10-04,transporte,Uber *Uber *Trip,28.99
28,2021-10-04,transporte,99app *99app,13.70
29,2021-10-04,transporte,Uber *Trip Help.Uber.C,4.50
33,2021-10-06,transporte,99app *99app,14.30
34,2021-10-06,transporte,99app *99app,13.40
38,2021-10-08,transporte,Uber *Trip,30.91


### > Save editted costs

In [38]:
nubank.to_csv('custos/custos-{}.csv'.format(month))